In [1]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load The Sign Language Gesture Images Dataset

Download: https://www.kaggle.com/ahmedkhanak1995/sign-language-gesture-images-dataset

In [2]:
# Sort the data folders in alphabetical order
dataFolders = sorted(os.listdir('SignLanguageRecognitionData/'))

# Delete .DS_Store if present
if dataFolders[0] == '.DS_Store':
    dataFolders.pop(0)
    
print(dataFolders)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [3]:
# Initialize lists for all images and labels in the dataset
images = []
labels = []

# Loop through all data folders
for i, label in enumerate(dataFolders):
    
    # Loop through all images in the data folder
    for j in os.listdir('SignLanguageRecognitionData/' + label):
        
        # Resize the image
        image = cv2.resize(cv2.imread('SignLanguageRecognitionData/' + label + '/' + j), (50,50))
        
        # Append the image and label to its respective list
        images.append(image)
        labels.append(i)

# Convert the images and labels lists to arrays
images = np.asarray(images)
labels = np.asarray(labels)

# Print the shape of the images and labels arrays
print(images.shape)
print(labels.shape)

(36000, 50, 50, 3)
(36000,)


# Split The Dataset Into Training, Validation, & Testing Data

In [4]:
# Split the dataset into 80% training data, 10% validation data, & 10% testing data
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size=0.2)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [5]:
# Print the shape of the training, validation, and testing data
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)
print(X_test.shape, Y_test.shape)

(28800, 50, 50, 3) (28800,)
(3600, 50, 50, 3) (3600,)
(3600, 50, 50, 3) (3600,)


In [6]:
# Convert Y_train, Y_val, & Y_test to One-Hot Encoded Vectors
Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)
Y_test = to_categorical(Y_test)
X_train = X_train / 255.
X_val = X_val / 255.
X_test = X_test / 255.

# Build The Sign Language Recognition Model

In [7]:
# Build the Sign Language Recognition model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
    tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
    tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
    tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
    tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(24, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 32)        9248      
                                                        

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train The Model

In [9]:
# Train the model
model.fit(X_train, Y_train, epochs=10, verbose=1, validation_data=(X_val, Y_val))

Epoch 1/10
900/900 [==============================] - 128s 141ms/step - loss: 0.3730 - accuracy: 0.8841 - val_loss: 0.0266 - val_accuracy: 0.9917
Epoch 2/10
900/900 [==============================] - 131s 146ms/step - loss: 0.0245 - accuracy: 0.9928 - val_loss: 0.0112 - val_accuracy: 0.9958
Epoch 3/10
900/900 [==============================] - 136s 151ms/step - loss: 0.0392 - accuracy: 0.9901 - val_loss: 0.0070 - val_accuracy: 0.9981
Epoch 4/10
900/900 [==============================] - 140s 156ms/step - loss: 0.0144 - accuracy: 0.9959 - val_loss: 0.0048 - val_accuracy: 0.9989
Epoch 5/10
900/900 [==============================] - 132s 147ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 9.7028e-06 - val_accuracy: 1.0000
Epoch 6/10
900/900 [==============================] - 127s 141ms/step - loss: 6.6835e-06 - accuracy: 1.0000 - val_loss: 3.4995e-06 - val_accuracy: 1.0000
Epoch 7/10
900/900 [==============================] - 131s 146ms/step - loss: 2.7319e-06 - accuracy: 1.0000 - va

# Test The Model

In [10]:
# Test the model
model.evaluate(X_test, Y_test)

113/113 [==============================] - 1s 11ms/step - loss: 3.6237e-07 - accuracy: 1.0000


[3.623691497978143e-07, 1.0]

# Save The Model

In [11]:
# Save the model
model.save('model.h5')